In [71]:
from itertools import product
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVC, LinearSVC
from sklearn.preprocessing import (MaxAbsScaler, MinMaxScaler,
                                   PolynomialFeatures, RobustScaler,
                                   StandardScaler)
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.model_selection import (BaseCrossValidator, GridSearchCV, KFold,
                                     RandomizedSearchCV, StratifiedKFold,
                                     check_cv, train_test_split)
from sklearn.linear_model import (ARDRegression, BayesianRidge, ElasticNet,
                                  ElasticNetCV, Lars, Lasso, LassoLars,
                                  LinearRegression, LogisticRegression,
                                  LogisticRegressionCV,
                                  OrthogonalMatchingPursuit, Ridge)
from sklearn.ensemble import (GradientBoostingClassifier,
                              GradientBoostingRegressor,
                              RandomForestClassifier, RandomForestRegressor)
from sklearn.base import BaseEstimator, is_regressor
import econml
from econml.orf import DMLOrthoForest
# from econml.metalearners import SLearner, TLearner, XLearner
from econml.grf import CausalForest
from econml.dr import DRLearner
from econml.dml import CausalForestDML, KernelDML, LinearDML, SparseLinearDML
import sklearn.preprocessing
import sklearn.neural_network
import sklearn.linear_model
import sklearn.ensemble
import sklearn
import argparse
import logging
import os
import pdb
import numpy as np
import pandas as pd
import pickle
import utils
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

In [72]:
data, X, T, Y, true_ITE, true_ATE, true_ATE_stderr, is_discrete = utils.load_ihdp()
# X, X_val, T, T_val, Y, Y_val = train_test_split(X, T, Y, train_size=0.6,shuffle=True, random_state=123)
# X_val, X_test, T_val, T_test, Y_val, Y_test = train_test_split(X_val, T_val, Y_val, train_size=.5, shuffle=True, random_state=123)

During Cross-fitting

In [74]:
ci_estimator_list = ['dml', 'CausalForestDML']
model_y = [ElasticNetCV(alphas=1), RandomForestRegressor(), GradientBoostingRegressor(), MLPRegressor()]
model_t = [LogisticRegressionCV(max_iter=500), RandomForestClassifier(), GradientBoostingClassifier(), MLPClassifier()]

In [76]:
#Divide into k folds.
#For each fold, select a different model and find the best hyperparameter and get score+mse of estimators. Set cv=0 for estimator
k = 4
cv = KFold(n_splits=k, shuffle=True, random_state=123)

fold_models = {}
i = 0
for train_index, test_index in cv.split(X):
    X_train, T_train, Y_train = X.iloc[train_index], T.iloc[train_index], Y.iloc[train_index]
    X_test, T_test, Y_test = X.iloc[test_index], T.iloc[test_index], Y.iloc[test_index]
    current_model_y, current_model_t = model_y[i], model_t[i]
    print(current_model_y, type(current_model_y))
    est_score = {}
    est_mse = {}
    for ci in ci_estimator_list:
        #find scores + mse's of all estimators
        if ci == 'dml':
            est = LinearDML(model_y='linear', model_t='linear', random_state=123)
            est.fit(Y_train, T_train, X=X_train, W=None)
            te_pred = est.effect(X_test)
            est_score[ci] = est.score(Y_test, T_test, X_test)
            est_mse[ci] = np.mean((Y_test - te_pred)**2)
        # elif ci == 'CausalForestDML':
        #     est = CausalForestDML(model_y='linear', model_t='linear', random_state=123)
        #     est.fit(Y_train, T_train, X=X_train, W=None)
        #     te_pred = est.effect(X_test)
        #     est_score[ci] = est.score(Y_test, T_test, X_test)
        #     est_mse[ci] = np.mean((Y_test - te_pred)**2)
        
    fold_models[f'fold {i}'] = {'model_y' : current_model_y, 'model_t' : current_model_t, 'Scores' : est_score, 'Mse' : est_mse}
    i += 1
    # break

fold_models

ElasticNetCV(alphas=1) <class 'sklearn.linear_model._coordinate_descent.ElasticNetCV'>


No scoring value was given. Using default score method neg_mean_squared_error.
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.039e-01, tolerance: 9.819e-02
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.469e-01, tolerance: 9.819e-02
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.862e-01, tolerance: 9.819e-02
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.278e-01, tolerance: 9.819e-02
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

RandomForestRegressor() <class 'sklearn.ensemble._forest.RandomForestRegressor'>


Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.310e-01, tolerance: 1.034e-01
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.030e-01, tolerance: 1.034e-01
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.074e-01, tolerance: 1.034e-01
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.259e-01, tolerance: 1.034e-01
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.012e-01, tolerance: 1.034e-01
Objective did n

GradientBoostingRegressor() <class 'sklearn.ensemble._gb.GradientBoostingRegressor'>


Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.239e-01, tolerance: 1.083e-01
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.811e-01, tolerance: 1.083e-01
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.156e-01, tolerance: 1.083e-01
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.945e-01, tolerance: 1.083e-01
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.711e-01, tolerance: 1.083e-01
Objective did n

MLPRegressor() <class 'sklearn.neural_network._multilayer_perceptron.MLPRegressor'>


Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.410e-01, tolerance: 1.084e-01
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.699e-01, tolerance: 1.084e-01
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.239e-01, tolerance: 1.084e-01
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.344e-01, tolerance: 1.084e-01
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.524e-01, tolerance: 1.084e-01
Objective did n

{'fold 0': {'model_y': ElasticNetCV(alphas=1),
  'model_t': LogisticRegressionCV(max_iter=500),
  'Scores': {'dml': 1.2758011012345938},
  'Mse': {'dml': 6.976192656016275}},
 'fold 1': {'model_y': RandomForestRegressor(),
  'model_t': RandomForestClassifier(),
  'Scores': {'dml': 1.667421877386907},
  'Mse': {'dml': 9.512641125094254}},
 'fold 2': {'model_y': GradientBoostingRegressor(),
  'model_t': GradientBoostingClassifier(),
  'Scores': {'dml': 1.4785260603355568},
  'Mse': {'dml': 7.619487695410438}},
 'fold 3': {'model_y': MLPRegressor(),
  'model_t': MLPClassifier(),
  'Scores': {'dml': 1.2892388724560588},
  'Mse': {'dml': 8.541823628781652}}}